# 9장 비지도학습 2부

## 주요 내용

- 군집/군집화
- k-평균
- **DBSCAN**
- **가우시안 혼합**

## DBSCAN

* 연속적인 밀집 지역을 하나의 군집으로 설정.

### 사이킷런의 DBSCAN 모델

* 두 개의 하이퍼파라미터 사용
    * `eps`: $\varepsilon$-이웃 범위
        * 주어진 기준값 $\varepsilon$ 반경 내에 위치한 샘플
    * `min_samples`: $\varepsilon$ 반경 내에 위치하는 이웃의 수

#### 핵심샘플과 군집

* 핵심샘플: $\varepsilon$ 반경 내에 자신을 포함해서 `min-samples`개의 이웃을 갖는 샘플

* 군집: 핵심샘플로 이루어진 이웃들로 구성된 그룹

#### 이상치

* 핵심샘플이 아니면서 동시에 핵심샘플의 이웃도 아닌 샘플.

#### 예제

* 반달모양 데이터 활용

---
```python
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.05, min_samples=5)
dbscan.fit(X)
```
---

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/handson-ml3/master/jupyter-book/imgs/ch09/homl09-16.png" width="600"/></div>

### DBSCAN과 예측

* `predict()` 메서드 지원하지 않음.

* 이유: `KNeighborsClassifier` 등 보다 좋은 성능의 분류 알고리즘 활용 가능.

* 아래 코드: 핵심샘플 대상 훈련.

---
```python
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(dbscan.components_, dbscan.labels_[dbscan.core_sample_indices_])
```
---

* 이후 새로운 샘플에 대한 예측 가능
* 아래 그림은 새로운 4개의 샘플에 대한 예측을 보여줌.

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/handson-ml3/master/jupyter-book/imgs/ch09/homl09-17.png" width="450"/></div>

#### 이상치 판단

* 위 예제에서, 두 군집으로부터 일정거리 이상 떨어진 샘플을 이상치로 간주 가능.

* 예를 들어, 양편 끝쪽에 위치한 두 개의 샘플이 이상치로 간주될 수 있음.

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/handson-ml3/master/jupyter-book/imgs/ch09/homl09-17a.png" width="450"/></div>

### DBSCAN의 장단점

* 매우 간단하면서 매우 강력한 알고리즘.
    * 하이퍼파라미터: 단 2개

* 군집의 모양과 개수에 상관없음.

* 이상치에 안정적임.

* 군깁 간의 밀집도가 크게 다르면 모든 군집 파악 불가능.

#### 계산복잡도

* 시간복잡도: 약 $O(m\, \log m)$. 단, $m$은 샘플 수

* 공간복잡도: 사이킷런의 DBSCAN 모델은 $O(m^2)$의 메모리 요구.
    * `eps`가 커질 경우.

### 기타 군집 알고리즘

* 응집 군집(병합 군집, agglomerative clustering)
* BIRCH
* 평균-이동
* 유사도 전파
* 스펙트럼 군집

## 9.2 가우시안 혼합 모델

* 데이터셋이 여러 개의 혼합된 가우시안 분포를 따르는 샘플들로 구성되었다고 가정.

* 가우시안 분포 = 정규분포

#### 정규분포 소개

* 종 모양의 확률밀도함수를 갖는 확률분포

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/handson-ml3/master/jupyter-book/imgs/ch09/homl09-18.png" width="400"/></div>

#### 군집

* 하나의 가우시안 분포에서 생생된 모든 샘플들의 그룹
* 일반적으로 타원형 모양.

### 예제

* 아래 그림에서처럼 일반적으로 모양, 크기, 밀집도, 방향이 다름.
* 따라서 각 샘플이 어떤 정규분포를 따르는지를 파악하는 게 핵심.

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/handson-ml3/master/jupyter-book/imgs/ch09/homl09-13.png" width="600"/></div>

### GMM 활용

* 위 데이터셋에 `GaussianMixture` 모델 적용

* `n_components`: 군집수 지정

* `n_init`: 모델 학습 반복 횟수. 
    * 파라미터(평균값, 공분산 등)를 무작위로 추정한 후 수렴할 때까지 학습시킴.

---
```python
from sklearn.mixture import GaussianMixture

gm = GaussianMixture(n_components=3, n_init=10, random_state=42)
gm.fit(X)
```
---

* 아래 그림은 학습된 모델을 보여줌.
    * 군집 평균, 결정 경계, 밀도 등고선

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/handson-ml3/master/jupyter-book/imgs/ch09/homl09-19.png" width="500"/></div>

### GMM 모델 규제

* 특성수가 크거나, 군집수가 많거나, 샘플이 적은 경우 최적 모델 학습 어려움.
* 공분산(covariance)에 규제를 가해서 학습을 도와줄 수 있음.
    * `covariance_type` 설정.

#### covariance_type 옵션값

* full
    * 아무런 제한 없음.
    * 기본값임.

* spherical
    * 군집이 원형이라 가정. 
    * 지름(분산)은 다를 수 있음.

* diag
    * 어떤 타원형도 가능.
    * 단. 타원의 축이 좌표축과 평행하다고 가정.

* tied
    * 모든 군집의 동일 모양, 동일 크기, 동일 방향을 갖는다고 가정.

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/handson-ml3/master/jupyter-book/imgs/ch09/homl09-20.png" width="600"/></div>

### 가우시안 혼합 모델 활용: 이상치 탐지

* 밀도가 임곗값보다 낮은 지역에 있는 샘플을 이상치로 간주 가능.

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/handson-ml3/master/jupyter-book/imgs/ch09/homl09-21.png" width="500"/></div>

### 가우션 혼합모델 군집수 지정

* k-평균에서 사용했던 관성 또는 실루엣 점수 사용 불가.
    * 군집이 타원형일 때 값이 일정하지 않기 때문.

* 대신에 __이론적 정보 기준__ 을 최소화 하는 모델 선택 가능.

#### 이론적 정보 기준

* BIC: Bayesian information criterion

    $$ \log(m)\, p - 2 \log (\hat L)$$

* AIC: Akaike information criterion

    $$ 2\, p - 2 \log (\hat L)$$

* $m$: 샘플 수
* $p$: 모델이 학습해야 할 파라미터 수
* $\hat L$: 모델의 가능도 함수의 최댓값

* 학습해야 할 파라미터가 많을 수록 벌칙이 가해짐.
* 데이터에 잘 학습하는 모델일 수록 보상을 더해줌.

#### 군집수와 정보조건

* 아래 그림은 군집수 $k$와 AIC, BIC의 관계를 보여줌.
* $k=3$이 최적으로 보임.

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/handson-ml3/master/jupyter-book/imgs/ch09/homl09-22.png" width="600"/></div>

### 베이즈 가우시안 혼합 모델

* 베이즈 확률통계론 활용

#### BayesianGaussianMixture 모델

* 최적의 군집수를 자동으로 찾아줌.
* 단, 최적의 군집수보다 큰 수를 `n_components`에 전달해야 함.
    * 즉, 군집에 대한 최소한의 정보를 알고 있다고 가정.
* 자동으로 불필요한 군집 제거

---
```python
from sklearn.mixture import BayesianGaussianMixture

bgm = BayesianGaussianMixture(n_components=10, n_init=10, random_state=42)
bgm.fit(X)
```
---

* 결과는 군집수 3개를 사용한 이전 결과와 거의 동일.
* 군집수 확인 가능

```python
>>> np.round(bgm.weights_, 2)
array([0.4 , 0.21, 0.4 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ])
```

#### 사전 믿음

* 군집수가 어느 정도일까를 나타내는 지수
* `weight_concentration_prior` 하이퍼파라미터
    * `n_components`에 설정된 군집수에 대한 규제로 사용됨.
    * 작은 값이면 특정 군집의 가중치를 0에 가깝게 만들어 군집수를 줄이도록 함.
    * 즉, 큰 값일 수록 `n_components`에 설정된 군집수가 유지되도록 함.

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/handson-ml3/master/jupyter-book/imgs/ch09/homl09-24.png" width="600"/></div>

### 가우시안 혼합 모델의 장단점

* 타원형 군집에 잘 작동.

* 하지만 다른 모양을 가진 데이터셋에서는 성능 좋지 않음.

* 예제: 달모양 데이터에 적용하는 경우
    * 억지로 타원을 찾으려 시도함.

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/handson-ml3/master/jupyter-book/imgs/ch09/homl09-23.png" width="700"/></div>

### 이상치 탐지와 특이치 탐지를 위한 다른 알고리즘

* PCA
* Fast-MCD
* 아이솔레이션 포레스트
* LOF
* one-class SVM